In [90]:
from tvDatafeed import TvDatafeed, Interval
from datetime import datetime

tv = TvDatafeed()

In [91]:
# futures NQ and ES as list
futures = ["NQ1!", "ES1!"]
# periods enum and short string list
periods = [[Interval.in_1_minute, "1M"],
            [Interval.in_5_minute, "5M"],
            # [Interval.in_30_minute, "30M"],
            # [Interval.in_4_hour, "4H"],
            # [Interval.in_daily,"1D"]
            ]
# print(periods[3][1])
periods[1][1]

'5M'

In [92]:
# this design make it more flexible to add more symbol and interval.
# merge them to new list
merged = [[future] + period for future in futures for period in periods]
print(merged)

[['NQ1!', <Interval.in_1_minute: '1'>, '1M'], ['NQ1!', <Interval.in_5_minute: '5'>, '5M'], ['ES1!', <Interval.in_1_minute: '1'>, '1M'], ['ES1!', <Interval.in_5_minute: '5'>, '5M']]


In [93]:
df = tv.get_hist(symbol='NQ1!', exchange='CME_MINI', interval=Interval.in_1_minute, n_bars=100000)
print(df)

                            symbol      open      high       low     close  \
datetime                                                                     
2024-08-22 20:06:00  CME_MINI:NQ1!  19645.00  19648.00  19644.00  19646.75   
2024-08-22 20:07:00  CME_MINI:NQ1!  19646.25  19647.50  19646.25  19647.50   
2024-08-22 20:08:00  CME_MINI:NQ1!  19648.25  19649.50  19645.25  19649.00   
2024-08-22 20:09:00  CME_MINI:NQ1!  19648.75  19653.00  19648.25  19652.50   
2024-08-22 20:10:00  CME_MINI:NQ1!  19652.75  19653.00  19650.75  19651.50   
...                            ...       ...       ...       ...       ...   
2024-09-08 22:23:00  CME_MINI:NQ1!  18522.50  18525.00  18522.50  18523.50   
2024-09-08 22:24:00  CME_MINI:NQ1!  18524.00  18525.00  18523.00  18524.50   
2024-09-08 22:25:00  CME_MINI:NQ1!  18525.00  18525.25  18522.75  18524.00   
2024-09-08 22:26:00  CME_MINI:NQ1!  18523.75  18524.50  18521.50  18523.25   
2024-09-08 22:27:00  CME_MINI:NQ1!  18524.00  18524.00  18524.00

In [9]:
import pandas as pd
from sqlalchemy import create_engine, inspect

engine = create_engine('sqlite:///data/ESNQ_DB.db')

In [10]:
table_names = inspect(engine).get_table_names()

print("Tables in the database:")
table_names

Tables in the database:


['ES1D',
 'ES1M',
 'ES30M',
 'ES4H',
 'ES5M',
 'NQ1D',
 'NQ1M',
 'NQ30M',
 'NQ4H',
 'NQ5M']

In [ ]:
files = ['NQ_2024-08-29_2024-09-13_1M.csv',
        'NQ_2024-08-18_2024-09-13_5M.csv',
        'NQ_2024-01-01_2024-09-13_30M.csv',
        'NQ_2021-01-03_2024-09-13_4H.csv',
        'NQ_1999-06-29_2024-09-12_1D.csv',
        'ES_2024-08-29_2024-09-13_1M.csv',
        'ES_2024-08-18_2024-09-13_5M.csv',
        'ES_2022-10-18_2024-09-13_30M.csv',
        'ES_2021-01-03_2024-09-13_4H.csv',
        'ES_1997-09-08_2024-09-12_1D.csv']

files


In [ ]:
for file in files:
    print(file)


In [19]:
def sql_importer (df, table_name):
    # Use inspector to check if the table exists
    inspector = inspect(engine)
    if not inspector.has_table(table_name):
        df.to_sql(table_name, engine, if_exists='append')
        print (f'New table created for {table_name} with {str(len(df))} rows')
    else:
        max_date = pd.read_sql(f'SELECT MAX(datetime) FROM {table_name}', engine).values[0][0]
        print(max_date)
        df = df[df.datetime > max_date]
        df.to_sql(table_name, engine, if_exists='append')
        print(str(len(df)) + ' new rows imported to DB')

In [ ]:
# Process the array to extract instrument and time frame
tablenames = [f"{file.split('_')[0]}{file.split('_')[-1].replace('.csv', '')}" for file in files]
tablenames

In [17]:
import pandas as pd
from sqlalchemy import create_engine
# merged = zip(files, tablenames)
merged = list(zip(files, tablenames))


In [20]:
import os
datadir = 'stockdata'
for file, tablename in merged:
    print(f"Loading data into {tablename}")
    filename = os.path.join(datadir, file)
    df = pd.read_csv(filename)
    sql_importer(df, tablename)

Loading data into NQ1M
2024-09-06 15:59:00
6899 new rows imported to DB
Loading data into NQ5M
2024-09-06 15:55:00
1380 new rows imported to DB
Loading data into NQ30M
2024-09-06 15:30:00
230 new rows imported to DB
Loading data into NQ4H
2024-09-06 13:00:00
30 new rows imported to DB
Loading data into NQ1D
2024-09-05 17:00:00
5 new rows imported to DB
Loading data into ES1M
2024-09-06 15:59:00
6900 new rows imported to DB
Loading data into ES5M
2024-09-06 15:55:00
1380 new rows imported to DB
Loading data into ES30M
2024-09-06 15:30:00
230 new rows imported to DB
Loading data into ES4H
2024-09-06 13:00:00
30 new rows imported to DB
Loading data into ES1D
2024-09-05 17:00:00
5 new rows imported to DB


In [21]:

# read from database and test
pd.read_sql('ES1M', engine)

,index,datetime,symbol,open,high,low,close,volume
0,0,2024-06-02 17:00:00,CME_MINI:NQ1!,18590.25,18598.50,18576.00,18595.25,876.0
1,1,2024-06-02 17:01:00,CME_MINI:NQ1!,18594.25,18597.00,18587.75,18587.75,350.0
2,2,2024-06-02 17:02:00,CME_MINI:NQ1!,18588.50,18588.50,18562.25,18564.50,594.0
3,3,2024-06-02 17:03:00,CME_MINI:NQ1!,18563.75,18567.50,18557.25,18560.00,573.0
4,4,2024-06-02 17:04:00,CME_MINI:NQ1!,18560.50,18564.50,18553.75,18553.75,209.0
...,...,...,...,...,...,...,...,...
89941,15075,2024-09-13 15:55:00,CME_MINI:ES1!,5627.00,5627.25,5626.25,5626.25,247.0
89942,15076,2024-09-13 15:56:00,CME_MINI:ES1!,5626.25,5626.75,5626.25,5626.50,79.0
89943,15077,2024-09-13 15:57:00,CME_MINI:ES1!,5626.75,5627.00,5626.75,5626.75,62.0
89944,15078,2024-09-13 15:58:00,CME_MINI:ES1!,5627.00,5627.50,5626.75,5627.25,279.0
